<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/7%20-%20Topic%20modelling/topic_modelling_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic modelling

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SYBGZL

In [14]:
!wget https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv

--2019-04-16 16:05:50--  https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54096356 (52M) [text/plain]
Saving to: ‘abcnews-date-text.csv’

abcnews-date-text.c 100%[===================>]  51.59M   160MB/s    in 0.3s    

2019-04-16 16:05:51 (160 MB/s) - ‘abcnews-date-text.csv’ saved [54096356/54096356]



In [22]:
import pandas as pd

headlines_df = pd.read_csv("abcnews-date-text.csv")
headlines_df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [16]:
headlines_df = headlines_df.sample(frac=.1, random_state=0)
headlines_df.shape

(108217, 2)

## Preprocess

In [35]:
import gensim
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:

lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(text):
  
  tokens = []
  
  for token in gensim.utils.simple_preprocess(text):
    if(token not in gensim.parsing.preprocessing.STOPWORDS):
      tokens.append(lemmatizer.lemmatize(token, pos='v'))
  
  return tokens

In [39]:
headlines_df["headline_processed"] = headlines_df["headline_text"].map(preprocess)
headlines_df.head()

NameError: ignored

In [41]:
headlines = headlines_df["headline_processed"].values
headlines[:3]

array([list(['aba', 'decide', 'community', 'broadcast', 'licence']),
       list(['act', 'witness', 'aware', 'defamation']),
       list(['call', 'infrastructure', 'protection', 'summit'])],
      dtype=object)

In [0]:
dictionary = gensim.corpora.Dictionary(headlines)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## Bag of Words

In [45]:
X = [dictionary.doc2bow(headline) for headline in headlines]
X[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [0]:
lda = gensim.models.LdaMulticore(X, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [49]:
for index, topic in lda.print_topics(-1):
    print("\nTOPIC %d - parole più popolari" % (index+1))
    print(topic)


TOPIC 1 - parole più popolari
0.033*"trump" + 0.029*"queensland" + 0.012*"close" + 0.011*"force" + 0.011*"island" + 0.010*"john" + 0.009*"david" + 0.008*"million" + 0.008*"christmas" + 0.007*"sea"

TOPIC 2 - parole più popolari
0.019*"win" + 0.015*"year" + 0.015*"interview" + 0.011*"world" + 0.011*"break" + 0.011*"league" + 0.010*"afl" + 0.010*"end" + 0.010*"donald" + 0.009*"final"

TOPIC 3 - parole più popolari
0.039*"man" + 0.034*"police" + 0.023*"charge" + 0.021*"court" + 0.016*"murder" + 0.015*"attack" + 0.015*"kill" + 0.014*"crash" + 0.013*"woman" + 0.013*"die"

TOPIC 4 - parole più popolari
0.018*"sa" + 0.017*"country" + 0.017*"rural" + 0.016*"fund" + 0.016*"wa" + 0.016*"help" + 0.015*"health" + 0.011*"indigenous" + 0.010*"nsw" + 0.009*"centre"

TOPIC 5 - parole più popolari
0.023*"south" + 0.020*"house" + 0.018*"north" + 0.015*"china" + 0.014*"west" + 0.014*"australia" + 0.012*"australian" + 0.011*"new" + 0.011*"turnbull" + 0.010*"talk"

TOPIC 6 - parole più popolari
0.025*"gov

## TF-IDF

In [0]:
tfidf = gensim.models.TfidfModel(X)
X = tfidf[X]

In [0]:
lda = gensim.models.LdaMulticore(X, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [54]:
for index, topic in lda.print_topics(-1):
    print("\nTOPIC %d - parole più popolari" % (index+1))
    print(topic)


TOPIC 1 - parole più popolari
0.014*"rural" + 0.013*"market" + 0.011*"news" + 0.009*"podcast" + 0.009*"abc" + 0.008*"price" + 0.007*"share" + 0.006*"turnbull" + 0.006*"queensland" + 0.006*"rise"

TOPIC 2 - parole più popolari
0.006*"coal" + 0.005*"capital" + 0.005*"hill" + 0.005*"hobart" + 0.005*"andrew" + 0.004*"fix" + 0.004*"western" + 0.004*"data" + 0.004*"exchange" + 0.003*"new"

TOPIC 3 - parole più popolari
0.006*"monday" + 0.006*"july" + 0.006*"mental" + 0.005*"islamic" + 0.005*"say" + 0.005*"northern" + 0.004*"australia" + 0.004*"png" + 0.004*"state" + 0.003*"clarke"

TOPIC 4 - parole più popolari
0.015*"trump" + 0.012*"interview" + 0.009*"grandstand" + 0.007*"nrl" + 0.007*"royal" + 0.006*"afl" + 0.006*"tuesday" + 0.006*"september" + 0.005*"commission" + 0.004*"malcolm"

TOPIC 5 - parole più popolari
0.007*"christmas" + 0.007*"august" + 0.007*"november" + 0.006*"april" + 0.006*"syria" + 0.006*"age" + 0.006*"street" + 0.006*"jam" + 0.004*"chris" + 0.004*"online"

TOPIC 6 - paro

## Visualizzare il modello

In [55]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 8.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


In [57]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, X, dictionary, mds='tsne')

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      -75.674393   57.601913       1        1  13.693484
7      -32.654701 -119.089577       2        1  11.061133
0      -86.308319  -31.780380       3        1  10.760223
8       -7.577353  -46.746807       4        1   9.962660
5       -3.299406   28.583500       5        1   9.914999
9       94.423058   59.448875       6        1   9.368862
1      132.890213  -35.508640       7        1   9.214301
3       16.216301  104.068535       8        1   8.987041
2       61.687859 -100.284302       9        1   8.867968
4       59.763020  -12.844320      10        1   8.169328, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
298    Default  7460.000000         man  7460.000000  30.0000  30.0000
212    Default  3663.000000       crash  3663.000000  29.0000  29.0000
458    Default  4501.000000     country  4501.000000  28.0000  28.0000
4560   Default  3987.000000        hour  3987.000000  27.0000  27.0000
7903   Default  3053.000000       trump  3053.000000  26.0000  26.0000
348    Default  4990.000000      charge  4990.000000  25.0000  25.0000
326    Default  3511.000000      murder  3511.000000  24.0000  24.0000
1394   Default  4547.000000       rural  4547.000000  23.0000  23.0000
1221   Default  3506.000000      market  3506.000000  22.0000  22.0000
1397   Default  2651.000000        news  2651.000000  21.0000  21.0000
1451   Default  4973.000000   interview  4973.000000  20.0000  20.0000
256    Default  6912.000000      police  6912.000000  19.0000  19.0000
14632  Default  2201.000000     podcast  2201.000000  18.0000  18.0000
2728   Default  4494.000000  government  4494.000000  17.0000  17.0000
3244   Default  1941.000000        drum  1941.000000  16.0000  16.0000
609    Default  2842.000000         car  2842.000000  15.0000  15.0000
6283   Default  1829.000000  grandstand  1829.000000  14.0000  14.0000
5081   Default  2402.000000         abc  2402.000000  13.0000  13.0000
732    Default  1842.000000     weather  1842.000000  12.0000  12.0000
1203   Default  3198.000000       woman  3198.000000  11.0000  11.0000
16928  Default  1816.000000         nrn  1816.000000  10.0000  10.0000
1298   Default  1964.000000      league  1964.000000   9.0000   9.0000
875    Default  1843.000000      guilty  1843.000000   8.0000   8.0000
896    Default  2292.000000       price  2292.000000   7.0000   7.0000
587    Default  3421.000000        kill  3421.000000   6.0000   6.0000
232    Default  4307.000000       court  4307.000000   5.0000   5.0000
1710   Default  1997.000000     assault  1997.000000   4.0000   4.0000
1171   Default  1599.000000       sport  1599.000000   3.0000   3.0000
5077   Default  1491.000000        john  1491.000000   2.0000   2.0000
2447   Default  1661.000000         nrl  1661.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
1430   Topic10   478.608734       coffs   479.467316   2.5030  -5.9709
6039   Topic10   469.622253     scandal   470.479858   2.5030  -5.9898
3476   Topic10   437.540192      voters   438.397491   2.5028  -6.0606
4022   Topic10   439.883820       basin   440.757629   2.5028  -6.0552
738    Topic10   431.095123       dream   431.988159   2.5027  -6.0754
6975   Topic10   397.735016         eat   398.595398   2.5026  -6.1559
3240   Topic10   393.430695      tennis   394.292450   2.5026  -6.1668
1396   Topic10   379.399536        hide   380.256958   2.5025  -6.2031
7946   Topic10   375.992920       colin   376.850647   2.5025  -6.2122
325    Topic10   369.943298        jury   370.801086   2.5025  -6.2284
14165  Topic10   364.031891      titans   364.889099   2.5024  -6.2445
5663   Topic10   569.257812        bike   571.705322   2.5005  -5.7974
2303   Topic10   422.826996      muslim   423.877380   2.5023  -6.